# hara-rating-bot
- HARA(Hazard Analysis &amp; Risk Assessment) rating bot HARA 수행 자동화 AI Bot 이며 나중에는 Model 구현(목표)
- 동작예시
```
[입력]
미션오일 누유 발생상황, 누출량은 시간당 0.1 mL/Hour 로 매우 적은 상황이며 현재 미션오일량은 충분한 상태

[답변]
도메인데이터셋 기반 답변이에요
Severity(위험도)=S0: 미션오일 누유가 발생하더라도 누출량이 매우 적고 현재 미션오일량이 충분한 상태이므로 부상의 위험은 없습니다.
Exposure(빈번도)=E1: 이러한 누유 상황은 매우 낮은 확률로 발생할 수 있습니다.
Controllability(통제도)=C0: 미션오일 누유는 일반적으로 운전자가 쉽게 제어할 수 있는 상황입니다.
QM (No Risk)
```

## 회고

### max_tokens 이슈
- pdf 파일 여러개를 머지해서 도큐먼트를 만들면서 사이드이팩트로 청크가 너무 커지는 버그가 있었고, 이로인해 openapi 호출할때 max_token 을 넘는 문제가 발생하였습니다.
- 이슈 해결을 위해 faiss_store 저장직전에 chunk_documents 사이즈를 한번 더 검증해서 벡터DB 에 집어넣도록 하여 해결하였습니다. (10번쨰 코드블럭에 마크다운으로 before & after 코드 추가작성 해두었습니다.
### openAI APIKEY 파일에서 읽어오는 방식으로 수정
```
OPENAI_APIKEY=""
with open("../secrets/apikey.txt", "r") as file:
    OPENAI_APIKEY = file.read().strip()  # 파일 내용을 읽고 공백 제거
os.environ['OPENAI_API_KEY'] =  OPENAI_APIKEY
```
- 노트북파일에 하드코딩해두지 않아도 키값 사용가능
- 까먹고 업로드하는경우가 많으므로 실수를 방지함

### 고품질 데이터셋 
- nhtsa (도로교통안전국) 데이터셋 주로 사용하였으며, 그 이유는, data.gov(연방 데이터셋) 일부 사용시도
### pdf 파일 다운로드 되지 않는 이슈 work-around
- nhtsa (도로교통안전국) 웹페이지 pdf 파일 링크의 경우 크롬브라우저 접근시 정상적으로 다운로드 가능하였으나, wget 혹은 python request 로 https/http 리퀘스트에 대하여 응답하지 않음.
- 크롤러/Bot 등으로 인식하는것으로 추청됨. User-Agent 를 헤더값에 추가하는 등 브라우저와 유사하게 시도해보았으나 봇판단 알고리즘이 고도화되어있어 실패
- 일일히 수동으로 pdf 파일 다운로드 후 파일을 옮겨 처리

In [1]:
!pip install langchain langchain-community openai gradio tiktoken faiss-cpu pypdf

In [2]:
!pip install -U langchain-openai
# 이슈 해결을 위해서 
"""
/var/folders/35/31j0q5116x1cqdmnp56bw1xc0000gp/T/ipykernel_86531/1353083462.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 객체 생성
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[8], line 3
      1 embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 객체 생성
      2 documents = merged_document
----> 3 faiss_store = FAISS.from_documents(documents, embeddings)
      4 retriever = faiss_store.as_retriever(search_kwargs={"k": 10})  # 상위 10개 검색
"""

'\n/var/folders/35/31j0q5116x1cqdmnp56bw1xc0000gp/T/ipykernel_86531/1353083462.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.\n  embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 객체 생성\n---------------------------------------------------------------------------\nAttributeError                            Traceback (most recent call last)\nCell In[8], line 3\n      1 embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 객체 생성\n      2 documents = merged_document\n----> 3 faiss_store = FAISS.from_documents(documents, embeddings)\n      4 retriever = faiss_store.as_retriever(search_kwargs={"k": 10})  # 상위 10개 검색\n'

In [3]:
# API 키를 상수가 아니라 apikey.txt 파일에 저장해놓고 읽어오는 방식으로 처리
OPENAI_APIKEY=""
with open("../secrets/apikey.txt", "r") as file:
    OPENAI_APIKEY = file.read().strip()  # 파일 내용을 읽고 공백 제거
# print(OPENAI_APIKEY)

In [4]:
import os
os.environ['OPENAI_API_KEY'] =  OPENAI_APIKEY

In [5]:
import numpy as np
import gradio as gr  # 웹 인터페이스 생성을 위한 Gradio 임포트

from langchain.chat_models import ChatOpenAI  # ChatGPT API 호출용 LLM 객체를 불러옴
from langchain.prompts import PromptTemplate  # 프롬프트 템플릿 생성을 위한 모듈
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings  # 문서를 벡터로 변환할 임베딩 모델
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 긴 텍스트를 일정 길이로 분할

from langchain_core.chat_history import BaseChatMessageHistory  # 대화 기록의 기본 클래스
from langchain_core.messages import BaseMessage  # 메시지 기본 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 히스토리와 함께 실행하는 체인 래퍼
from langchain_community.vectorstores import FAISS # 벡터 데이터베이스 FAISS

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import CrossEncoder
from langchain.schema.runnable import RunnableLambda

from typing import List
import requests
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 긴 텍스트를 일정 길이로 분할
import pandas as pd
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings


In [6]:
# 데이터셋 출처 
"""
1. 파이썬 다운로드가 실패하는 이슈 있었습니다.
2. 웹브라우저로 다운시 잘 받아지지만, 다른 방법인 wget 이나 python request 로 접근시 bot으로 판단하여 request 무응답으로 처리합니다.
3. 이에대한 해결책을 그냥 버튼을 눌러 다운로드 한다음 pdf 파일을 업로드하는 방식으로 처리하였습니다.
4. 데이터의 출처는 : nhtsa(미국 도로교통국, ISO26262 국제표준문서, WIKI 백과 등 입니다
data origins
    "https://www.nhtsa.gov/sites/nhtsa.gov/files/2021-06/Hazard%20Analysis%20of%20Concept%20Heavy%20Truck%20Platooning%20Systems-%20A.Svenson%20final_0.pdf",
    "https://www.nhtsa.gov/sites/nhtsa.gov/files/documents/13498a_812_573_alcsystemreport.pdf",
    "https://www.nhtsa.gov/sites/nhtsa.gov/files/812285_electronicsreliabilityreport.pdf",
    "https://www.nhtsa.gov/sites/nhtsa.gov/files/2015sae-hommes-safetyanalysisapproaches.pdf",
    "https://www.nhtsa.gov/sites/nhtsa.gov/files/documents/13502_812576_steerbywire.pdf",
    "https://www.nhtsa.gov/sites/nhtsa.gov/files/documents/13497a_812574_hydraulicbrakingsystem.pdf"
    "https://data.montgomerycountymd.gov/api/views/n7fk-dce5/rows.csv?accessType=DOWNLOAD",
    "https://data.tempe.gov/api/download/v1/items/0c333bd164d64d62aa0ee6f99b1ccf82/csv?layers=0",
    "https://data.townofcary.org/api/v2/catalog/datasets/cpd-crash-incidents/exports/csv",
    "https://data.transportation.gov/api/views/6axg-epim/rows.csv?accessType=DOWNLOAD",
    "https://data.montgomerycountymd.gov/api/views/mmzv-x632/rows.csv?accessType=DOWNLOAD",
    "https://data.cityofnewyork.us/api/views/knr6-vurn/rows.csv?accessType=DOWNLOAD",
    "https://data.montgomerycountymd.gov/api/views/n7fk-dce5/rows.csv?accessType=DOWNLOAD"
"""
## 데이터셋 
pdf_paths: [] = [
    "1.pdf",
    "2.pdf",
    "3.pdf",
    "4.pdf",
    "5.pdf",
    "6.pdf",
    "7.pdf",
    "8.pdf",
    "9.pdf",
    "10.pdf",
    "11.pdf",
    "12.pdf"
]

In [7]:

def load_pdf(pdf_path: str):
    try:
        print(f"PDF loading, path=[{pdf_path}]") # PyPDFLoader는 로컬 파일 경로를 필요로 하므로 다운로드한 파일 경로를 사용
        loader = PyPDFLoader(pdf_path)  # pdf 로더로 로딩
        pages = loader.load_and_split()  # 페이지로 쪼개기
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)  # 청크로 쪼개기
        documents = text_splitter.split_documents(pages)
        print(f"PDF 파일 적재 성공, len={len(documents)}")
        return documents
    except Exception as e:
        print(f"❌ PDF 로딩 중 오류 발생: {e}")
        return []  # 에러 발생 시 빈 리스트 반환




In [8]:
all_documents = []
print("dataset loading start")

for pdf_item in pdf_paths:
    all_documents.extend(load_pdf(pdf_item))

merged_document = Document(page_content=" ".join([doc.page_content for doc in all_documents if len(doc.page_content) < 1000]))




dataset loading start
PDF loading, path=[1.pdf]
PDF 파일 적재 성공, len=1476
PDF loading, path=[2.pdf]
PDF 파일 적재 성공, len=32
PDF loading, path=[3.pdf]
PDF 파일 적재 성공, len=237
PDF loading, path=[4.pdf]
PDF 파일 적재 성공, len=24
PDF loading, path=[5.pdf]
PDF 파일 적재 성공, len=674
PDF loading, path=[6.pdf]
PDF 파일 적재 성공, len=131
PDF loading, path=[7.pdf]
PDF 파일 적재 성공, len=14
PDF loading, path=[8.pdf]
PDF 파일 적재 성공, len=218
PDF loading, path=[9.pdf]
PDF 파일 적재 성공, len=16
PDF loading, path=[10.pdf]
PDF 파일 적재 성공, len=449
PDF loading, path=[11.pdf]
PDF 파일 적재 성공, len=51
PDF loading, path=[12.pdf]
PDF 파일 적재 성공, len=54


In [9]:
#print(merged_document)

In [10]:
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 객체 생성


#documents = [merged_document]
#faiss_store = FAISS.from_documents(documents, embeddings)
#retriever = faiss_store.as_retriever(search_kwargs={"k":5})  # 상위 1개 검색

def split_document_into_chunks(document, chunk_size=1000):
    # 문서 텍스트를 일정한 크기(여기서는 1000자)로 나누기
    chunks = [document[i:i+chunk_size] for i in range(0, len(document), chunk_size)]
    return chunks

# 각 청크를 임베딩 후 FAISS에 추가
chunks = split_document_into_chunks(merged_document.page_content)
chunk_documents = [Document(page_content=chunk) for chunk in chunks]
faiss_store = FAISS.from_documents(chunk_documents, embeddings)
retriever = faiss_store.as_retriever(search_kwargs={"k": 10})  #

# 🌟🌟🌟🌟🌟🌟트러블슈팅 포인트🌟🌟🌟🌟🌟
- pdf 파일 여러개를 머지해서 도큐먼트를 만드니까 청크가 너무 커서 openapi 호출할때 max_token 을 넘는 문제가 발생하였습니다.
## 수정 전
```
documents = [merged_document]
faiss_store = FAISS.from_documents(documents, embeddings)
retriever = faiss_store.as_retriever(search_kwargs={"k":5})  # 상위 1개 검색
```
## 수정 후
```
chunks = split_document_into_chunks(merged_document.page_content)
chunk_documents = [Document(page_content=chunk) for chunk in chunks]
faiss_store = FAISS.from_documents(chunk_documents, embeddings)
retriever = faiss_store.as_retriever(search_kwargs={"k": 5})  #
```


In [11]:
llm = ChatOpenAI(
    temperature=0.1,          # 생성 답변의 창의성 조절 (낮을수록 결정적)
    max_tokens=4000,          # 최대 생성 토큰 수
    model_name="gpt-4o", # 사용할 모델 이름
)

/var/folders/35/31j0q5116x1cqdmnp56bw1xc0000gp/T/ipykernel_88191/4063342559.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [12]:
#!pip install -U langchain-openai
#!export TOKENIZERS_PARALLELISM=false


In [13]:
prompt_template = """
-
너는 HARA(Hazard Analysis &amp; Risk Assessment) 점수평가 자동화(Rating) bot
Based on the reference document, if the content is found in the document, reply with 'LLM 기반 답변이에요' and provide the response.
If the content is not in the document, reply with "도메인데이터셋 기반 답변이에요" and provide the response.
-
Input Description:The input describes a potential situation a vehicle may encounter.
Example Inputs:
(Example 1)Cut-in by a side vehicle during low-speed driving.
---
ISO 26262 provides a standard for functional safety management for automotive applications, defining standards for overall organizational safety management as well as standards for a safety life cycle for the development and production of individual automotive products.[6][7][8][9] The ISO 26262 safety life cycle described in the next section operates on the following safety management concepts:[1]
Hazardous Event
A hazardous event is a relevant combination of a vehicle-level hazard and an operational situation of the vehicle with potential to lead to an accident if not controlled by timely driver action.
Safety Goal
A safety goal is a top-level safety requirement that is assigned to a system, with the purpose of reducing the risk of one or more hazardous events to a tolerable level.
Automotive Safety Integrity Level
An Automotive Safety Integrity Level (ASIL) represents an automotive-specific risk-based classification of a safety goal as well as the validation and confirmation measures required by the standard to ensure accomplishment of that goal.
Safety Requirement
Safety requirements include all safety goals and all levels of requirements decomposed from the safety goals down to and including the lowest level of functional and technical safety requirements allocated to hardware and software components.
-
Output Description

Severity Classifications (S):
S0 No Injuries
S1 Light to moderate injuries
S2 Severe to life-threatening (survival probable) injuries
S3 Life-threatening (survival uncertain) to fatal injuries
-
Exposure Classifications (E):
E0 Incredibly unlikely
E1 Very low probability (injury could happen only in rare operating conditions)
E2 Low probability
E3 Medium probability
E4 High probability (injury could happen under most operating conditions)
-
Controllability Classifications (C):
C0 Controllable in general
C1 Simply controllable
C2 Normally controllable, most drivers could act to prevent injury
C3 Difficult to control or uncontrollable
-
ASIL LEVEL
ASIL D:indicates a high risk of severe injury or fatality
ASIL C:Ex=Loss of rear wheel braking or cruise control failure. 
ASIL B:Ex=Headlights or brake lights failure. Can be modeled with informal languages, 
ASIL A:(Lowest Risk): Ex=Tail lights failure. Allows for less formal design inspections and walkthroughs during development
QM (No Risk): No automotive hazards; no safety requirements to manage under ISO 26262.
-
답변은 아래 형식으로 출력되어야함

Severity(위험도)=S1:@why
Exposure(빈번도)=E2:@why
Controllability(통제도)=C2:@why
ASIL B
-
이전 대화:{chat_history}
참고 문서:{context}
질문:{question}
답변:

모든 답변은 한국어로 답변해줘
"""
qa_prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [14]:
crossencoder = CrossEncoder('BAAI/bge-reranker-v2-m3')

# Faiss + Cross Encoder 기반 Retriever 함수 정의
def cross_encoder_retriever(question):
    """Faiss로 검색한 문서들을 Cross Encoder로 재정렬하여 최종 결과 반환"""
    retrieved_docs = retriever.invoke(question)  # Faiss 기반 검색
    retrieved_texts = [doc.page_content for doc in retrieved_docs]

    # Cross Encoder로 Reranking 수행
    query_doc_pairs = [(question, doc) for doc in retrieved_texts]
    cross_scores = crossencoder.predict(query_doc_pairs)

    # 상위 2개 선택
    best_indices = np.argsort(cross_scores)[::-1][:2]
    best_docs = [retrieved_texts[i] for i in best_indices]

    return "\n".join(best_docs)  # 최종 선택된 문서 반환

In [15]:
class LimitedChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, max_conversations: int = 2):
        self.full_messages: List[BaseMessage] = []  # 전체 메시지 저장 리스트
        self.messages: List[BaseMessage] = []       # 제한된 메시지 저장 리스트
        self.max_messages = (max_conversations - 1) * 2  # 최근 대화 쌍(사용자+AI)만 저장

    def add_message(self, message: BaseMessage) -> None:
        self.full_messages.append(message)  # 전체 기록에 추가
        self.messages.append(message)       # 제한된 기록에도 추가
        if len(self.messages) > self.max_messages:
            self.messages.pop(0)  # 제한 초과 시 가장 오래된 메시지 제거

    def clear(self) -> None:
        self.messages = []  # 대화 기록 초기화

    def get_messages(self) -> List[BaseMessage]:
        # 최근 대화 쌍을 포함해 반환 (추가로 과거 대화가 있다면 그 일부 포함)
        return self.full_messages[-self.max_messages-2:]

In [16]:
# message_store의 모든 세션 메시지 기록을 삭제
#message_store.clear()
message_store = {}
# 새로 세션을 만들거나, 초기화할 경우
session_id = "harabot_session"  # 새로운 세션 ID 지정
message_store[session_id] = LimitedChatMessageHistory(max_conversations=5)  # 빈 히스토리로 새 세션 생성

In [17]:
message_store

{'harabot_session': <__main__.LimitedChatMessageHistory at 0x333447010>}

In [18]:
# chat_history를 출력하기 위한 함수
def format_history(chat_history):
    formatted = ""
    for msg in chat_history:
        if msg.type == "human":
            formatted += f"사용자: {msg.content}\n"
        elif msg.type == "ai":
            formatted += f"모두봇: {msg.content}\n"
    return formatted

In [19]:
# LangChain Chain 설정 (Cross Encoder 적용)
chain = (
    {
        "context": itemgetter("question") | RunnableLambda(cross_encoder_retriever),
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history") | RunnableLambda(format_history),
    }
    | qa_prompt
    | llm
    | StrOutputParser()
)

In [20]:
chain_with_chat_history = RunnableWithMessageHistory(
    chain,
    lambda sid: message_store[sid],            # 세션 ID로 해당 채팅 히스토리 반환
    input_messages_key="question",             # 입력 데이터에서 질문을 추출하는 키
    history_messages_key="chat_history"        # 체인에 전달할 히스토리 키 이름
)

## 대화 테스트 (멀티턴 테스트)

## 트러블슈팅
```log
BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[0].content': string too long. Expected a string with maximum length 1048576, but got a string with length 1361161 instead.", 'type': 'invalid_request_error', 'param': 'messages[0].content', 'code': 'string_above_max_length'}}
```
### 해결책 : 입력 데이터 크기 줄이기
- 채팅 히스토리 줄이기 : chain_with_chat_history.invoke() 호출 시 전달하는 messages[0].content 크기를 줄이기
  - 채팅 히스토리 줄이기 : 2로 설정
  - 모든 세션의 히스토리 초기화 : message_store.clear()
  - 프롬프트 최대한 영어로 해서 토큰효율성 높이기 ★

In [21]:
print("Question:", {"question": "브레이크 고장"})
print("Session ID:", session_id)
print("Messages:", message_store[session_id].get_messages())
#print("messages[0].content:", message_store[session_id].get_messages()[0].content)
print("Before adding to prompt_template:", len(message_store[session_id].get_messages()))
formatted_history = format_history(message_store[session_id].get_messages())
print("After formatting history:", len(formatted_history))

Question: {'question': '브레이크 고장'}
Session ID: harabot_session
Messages: []
Before adding to prompt_template: 0
After formatting history: 0


In [22]:
result = chain_with_chat_history.invoke(
            {"question": "브레이크 고장"},
            config={"configurable": {"session_id": session_id}}
)

In [23]:
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S2: 브레이크 고장은 심각한 사고로 이어질 수 있으며, 이는 심각한 부상을 초래할 수 있습니다.
Exposure(빈번도)=E3: 브레이크 고장은 다양한 주행 조건에서 발생할 수 있으며, 이는 중간 정도의 빈도로 발생할 수 있습니다.
Controllability(통제도)=C3: 브레이크 고장은 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.
ASIL D


In [24]:
result = chain_with_chat_history.invoke(
            {"question": "방금 내 질문과 답변 다시 알려줘"},
            config={"configurable": {"session_id": session_id}}
)

In [25]:
print(result)

도메인데이터셋 기반 답변이에요

이전 대화에서의 질문과 답변은 다음과 같습니다:

질문: 브레이크 고장

답변:
Severity(위험도)=S2: 브레이크 고장은 심각한 사고로 이어질 수 있으며, 이는 심각한 부상을 초래할 수 있습니다.
Exposure(빈번도)=E3: 브레이크 고장은 다양한 주행 조건에서 발생할 수 있으며, 이는 중간 정도의 빈도로 발생할 수 있습니다.
Controllability(통제도)=C3: 브레이크 고장은 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.
ASIL D


In [26]:
result = chain_with_chat_history.invoke(
            {"question": "자동차 엔진 제어시스템 fail 엔진 제어 유닛(ECU) 오류로 인해 엔진이 갑자기 멈추거나 비정상적인 작동하는 상황"},
            config={"configurable": {"session_id": session_id}}
)
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S2: 엔진 제어 유닛(ECU) 오류로 인해 엔진이 갑자기 멈추거나 비정상적으로 작동하면 심각한 사고로 이어질 수 있으며, 이는 심각한 부상을 초래할 수 있습니다.
Exposure(빈번도)=E3: 엔진 제어 시스템의 오류는 다양한 주행 조건에서 발생할 수 있으며, 이는 중간 정도의 빈도로 발생할 수 있습니다.
Controllability(통제도)=C3: 엔진 제어 시스템의 오류는 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.
ASIL D


In [27]:
result = chain_with_chat_history.invoke(
            {"question": "ABS 시스템에 오류가 발생하여 브레이크 잠김 현상이 발생, 급브레이크 시 차량이 제어되지 않고 브레이크가 잠겨 적절한 제어 불능 상태"},
            config={"configurable": {"session_id": session_id}}
)
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S2: ABS 시스템 오류로 인해 브레이크 잠김 현상이 발생하면 심각한 사고로 이어질 수 있으며, 이는 심각한 부상을 초래할 수 있습니다.
Exposure(빈번도)=E3: ABS 시스템의 오류는 다양한 주행 조건에서 발생할 수 있으며, 이는 중간 정도의 빈도로 발생할 수 있습니다.
Controllability(통제도)=C3: ABS 시스템의 오류는 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.
ASIL D


In [28]:
result = chain_with_chat_history.invoke(
            {"question": "시속40km/h의 저속상황의 고속도로 진출입로의 차량이 많은곳에서 차선 유지 보조 시스템(LKA)이 오작동하여 차선 중앙을 벗어나거나 다른 차선으로 이동하는 상황"},
            config={"configurable": {"session_id": session_id}}
)
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S2: 차선 유지 보조 시스템(LKA)의 오작동으로 인해 차량이 차선 중앙을 벗어나거나 다른 차선으로 이동하면 심각한 사고로 이어질 수 있으며, 이는 심각한 부상을 초래할 수 있습니다.
Exposure(빈번도)=E3: LKA 시스템의 오작동은 다양한 주행 조건에서 발생할 수 있으며, 이는 중간 정도의 빈도로 발생할 수 있습니다.
Controllability(통제도)=C3: LKA 시스템의 오작동은 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.
ASIL D


In [29]:
result = chain_with_chat_history.invoke(
            {"question": "조수석방향에서 차량 전체폭의 22% 정도 지점을 장애물 시속 70km/h 속도 충격시 에어백 시스템이 충돌 시 에어백 시스템의 센서 인식범위 외 지점으로 인하여 시스템은 정상이나 작동하지 않는 상황"},
            config={"configurable": {"session_id": session_id}}
)
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S2: 에어백 시스템이 작동하지 않으면 충돌 시 심각한 부상을 초래할 수 있습니다.
Exposure(빈번도)=E2: 에어백 시스템의 센서 인식 범위 외 지점에서의 충돌은 낮은 빈도로 발생할 수 있습니다.
Controllability(통제도)=C3: 에어백 시스템이 작동하지 않는 상황은 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.
ASIL D


In [30]:
result = chain_with_chat_history.invoke(
            {"question": "미션오일 누유 발생상황, 누출량은 시간당 0.1 mL/Hour 로 매우 적은 상황이며 현재 미션오일량은 충분한 상태"},
            config={"configurable": {"session_id": session_id}}
)
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S0: 미션오일 누유가 발생하더라도 누출량이 매우 적고 현재 미션오일량이 충분한 상태이므로 부상의 위험은 없습니다.
Exposure(빈번도)=E1: 미션오일 누유는 드물게 발생할 수 있는 상황입니다.
Controllability(통제도)=C0: 미션오일 누유는 일반적으로 운전자가 쉽게 제어할 수 있는 상황입니다.
QM (No Risk)


In [31]:
result = chain_with_chat_history.invoke(
            {"question": "시속150km/h 이상의 고속주행시 트렁크 계폐여부 감지센서 고징 상황"},
            config={"configurable": {"session_id": session_id}}
)
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S1: 트렁크 계폐여부 감지센서의 고장은 고속주행 시 트렁크가 열릴 가능성을 증가시킬 수 있으며, 이는 경미한 부상을 초래할 수 있습니다.
Exposure(빈번도)=E2: 고속주행 시 트렁크 계폐여부 감지센서의 고장은 낮은 빈도로 발생할 수 있습니다.
Controllability(통제도)=C2: 트렁크가 열리는 상황은 보통의 경우 운전자가 제어할 수 있습니다.
ASIL B


In [32]:
result = chain_with_chat_history.invoke(
            {"question": "운전석으로 배기가스 유입 및 휘발유 등의 유증기 냄새 감지"},
            config={"configurable": {"session_id": session_id}}
)
print(result)

도메인데이터셋 기반 답변이에요

Severity(위험도)=S1: 배기가스 유입 및 휘발유 등의 유증기 냄새는 경미한 건강 문제를 초래할 수 있습니다.
Exposure(빈번도)=E2: 이러한 상황은 낮은 빈도로 발생할 수 있습니다.
Controllability(통제도)=C1: 대부분의 경우 운전자가 창문을 열거나 환기를 통해 쉽게 제어할 수 있습니다.
ASIL A


In [33]:
# 현재까지 기록되고 있는 모든 이전 대화를 출력
message_store['harabot_session'].get_messages()

[HumanMessage(content='시속40km/h의 저속상황의 고속도로 진출입로의 차량이 많은곳에서 차선 유지 보조 시스템(LKA)이 오작동하여 차선 중앙을 벗어나거나 다른 차선으로 이동하는 상황', additional_kwargs={}, response_metadata={}),
 AIMessage(content='도메인데이터셋 기반 답변이에요\n\nSeverity(위험도)=S2: 차선 유지 보조 시스템(LKA)의 오작동으로 인해 차량이 차선 중앙을 벗어나거나 다른 차선으로 이동하면 심각한 사고로 이어질 수 있으며, 이는 심각한 부상을 초래할 수 있습니다.\nExposure(빈번도)=E3: LKA 시스템의 오작동은 다양한 주행 조건에서 발생할 수 있으며, 이는 중간 정도의 빈도로 발생할 수 있습니다.\nControllability(통제도)=C3: LKA 시스템의 오작동은 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.\nASIL D', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='조수석방향에서 차량 전체폭의 22% 정도 지점을 장애물 시속 70km/h 속도 충격시 에어백 시스템이 충돌 시 에어백 시스템의 센서 인식범위 외 지점으로 인하여 시스템은 정상이나 작동하지 않는 상황', additional_kwargs={}, response_metadata={}),
 AIMessage(content='도메인데이터셋 기반 답변이에요\n\nSeverity(위험도)=S2: 에어백 시스템이 작동하지 않으면 충돌 시 심각한 부상을 초래할 수 있습니다.\nExposure(빈번도)=E2: 에어백 시스템의 센서 인식 범위 외 지점에서의 충돌은 낮은 빈도로 발생할 수 있습니다.\nControllability(통제도)=C3: 에어백 시스템이 작동하지 않는 상황은 운전자가 제어하기 어려운 상황을 초래할 수 있습니다.\nASIL D', additional_kwargs={}, response_metada

## 그라디오 데모

In [ ]:
import gradio as gr


# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="HARA Rating bot", type='messages')
    msg = gr.Textbox(label="질문해주세요!")
    clear = gr.Button("대화 초기화")

    def respond(message, chat_history):
        # 세션 ID와 함께 질문을 체인에 전달하여 결과 받기
        result = chain_with_chat_history.invoke(
            {"question": message},
            config={"configurable": {"session_id": session_id}}
        )

        # 대화 기록에 사용자와 봇의 메시지 추가
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": result})

        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: [], None, chatbot, queue=False)  # 대화 기록 초기화

demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
